<a href="https://colab.research.google.com/github/julija-dmrk/data-mining/blob/main/3rd_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suicide's data classification

In [1]:
import matplotlib.pyplot as plt   # visualization
import seaborn as sns             # visualization
import numpy as np                # data manipulation
import pandas as pd               # data manipulation, data processing, CSV file I/O (e.g. pd.read_csv
from sklearn import pipeline
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut, cross_val_score, cross_validate, GridSearchCV, train_test_split

In [2]:
url = "https://raw.githubusercontent.com/VitaT/ML-python/main/data/suicides.csv"
data = pd.read_csv(url)

In [ ]:
data.info()
data.head(5)
data.shape
data.sample(10)